<a href="https://colab.research.google.com/github/Nikhileswar-Komati/Deep-Learning/blob/master/horse_or_human_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

--2020-04-14 07:23:53--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.128, 2404:6800:4008:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M   104MB/s    in 1.4s    

2020-04-14 07:23:55 (104 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]



In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip

--2020-04-14 07:39:22--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.97.128, 2404:6800:4008:c00::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.97.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/zip]
Saving to: ‘/tmp/validation-horse-or-human.zip’

/tmp/validation-hor 100%[===================>]  10.95M  30.0MB/s    in 0.4s    

2020-04-14 07:39:23 (30.0 MB/s) - ‘/tmp/validation-horse-or-human.zip’ saved [11480187/11480187]



In [0]:
import os
import zipfile

zip_ref = zipfile.ZipFile('/tmp/horse-or-human.zip', 'r')
zip_ref.extractall('/tmp/horse-or-human')
zip_ref.close()

In [0]:
zips = zipfile.ZipFile('/tmp/validation-horse-or-human.zip', 'r')
zips.extractall('/tmp/validation-horse-or-human')
zips.close()

In [0]:
train_horses_dir = os.path.join('/tmp/horse-or-human/horses')
train_humans_dir = os.path.join('/tmp/horse-or-human/humans')
test_horses_dir = os.path.join('/tmp/validation-horse-or-human/horses')
test_humans_dir = os.path.join('/tmp/validation-horse-or-human/humans')

In [0]:
print('total training horse images:', len(os.listdir(train_horses_dir)))
print('total training human images:', len(os.listdir(train_humans_dir)))
print('total training horse images:', len(os.listdir(test_horses_dir)))
print('total training human images:', len(os.listdir(test_humans_dir)))

total training horse images: 500
total training human images: 527
total training horse images: 128
total training human images: 128


In [0]:
import tensorflow as tf

model = tf.keras.models.Sequential(
    [
     tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu', input_shape = (300, 300, 3)),
     tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
    ]
)

In [0]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 33, 33, 64)       

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen_obj = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen_obj.flow_from_directory(
        '/tmp/horse-or-human/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')
test_datagen_obj = ImageDataGenerator(rescale=1/255)

test_generator = test_datagen_obj.flow_from_directory(
        '/tmp/validation-horse-or-human/', 
        target_size = (300,300), 
        batch_size = 32, 
        class_mode = 'binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [0]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = test_generator,
      validation_steps=8)

Epoch 1/15
8/8 [==============================] - 6s 808ms/step - loss: 1.1839 - accuracy: 0.5195 - val_loss: 0.6522 - val_accuracy: 0.8516
Epoch 2/15
8/8 [==============================] - 7s 831ms/step - loss: 0.6772 - accuracy: 0.7188 - val_loss: 1.2283 - val_accuracy: 0.5000
Epoch 3/15
8/8 [==============================] - 7s 871ms/step - loss: 0.6583 - accuracy: 0.7297 - val_loss: 2.5579 - val_accuracy: 0.4961
Epoch 4/15
8/8 [==============================] - 6s 777ms/step - loss: 0.8610 - accuracy: 0.7141 - val_loss: 1.0678 - val_accuracy: 0.7266
Epoch 5/15
8/8 [==============================] - 6s 775ms/step - loss: 0.4224 - accuracy: 0.8276 - val_loss: 0.9674 - val_accuracy: 0.8672
Epoch 6/15
8/8 [==============================] - 6s 774ms/step - loss: 0.5328 - accuracy: 0.8154 - val_loss: 0.5649 - val_accuracy: 0.8242
Epoch 7/15
8/8 [==============================] - 6s 773ms/step - loss: 0.2691 - accuracy: 0.8943 - val_loss: 0.7274 - val_accuracy: 0.8711
Epoch 8/15
8/8 [====

In [0]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")
 

Saving horse01-0.png to horse01-0.png
Saving images.jpg to images.jpg
[0.]
horse01-0.png is a horse
[0.]
images.jpg is a horse
